<a href="https://colab.research.google.com/github/DevKadyn/Stable-Diffusion/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup + Settings


In [ ]:
#@markdown ###Save_Path
#@markdown Save_Path will save at the root of your Google Drive. Don't start or end your Save_Path with "/"
Save_Path = "" #@param {type:"string"}
#@markdown ###HF_Token
#@markdown HF_Token will automatically connect you to HuggingFace. You can 
HF_Token = "" #@param {type:"string"}
#@markdown ###Precision
#@markdown 
Precision = "Higher Precision [High Ram]" #@param ["Lower Precision [Low Ram]", "Higher Precision [High Ram]"]
#@markdown ###Schedule
#@markdown 
Schedule = "K-LMS" #@param ["PNDMS", "DDIM", "K-LMS"]


#Import Libraries
print("Lucid Dreams: Installing libraries...")

import time
import os
import os.path
from google.colab import output
from IPython.display import clear_output 
output.enable_custom_widget_manager()

!pip install diffusers==0.3.0 transformers ftfy --quiet
!pip install transformers scipy ftfy --quiet
!pip install "ipywidgets>=7,<8" --quiet

from google.colab import output
from IPython.display import clear_output 
output.clear()

#Start GPU
import subprocess
nvidiasmi_output = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Lucid Dreams:" + str(nvidiasmi_output))
time.sleep(2)
output.clear()

#Mount Google Drive
save_path = "003 - Creative/002 - Personal/AI Generated/Lucid Diffusion"
from google.colab import drive 
drive.mount('/content/drive')
root_path = '/content/drive/MyDrive/' + save_path
if not os.path.exists(root_path):
  os.mkdir(root_path)
print("Lucid Dreams: Mounted Google Drive")
time.sleep(2)
output.clear()

print("Lucid Dreams has installed it's libraries")
time.sleep(2)
output.clear()

# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

print("Lucid Dreams: Installed upscaler")
time.sleep(2)
output.clear()

#Authenticate Huggingface
from huggingface_hub.commands.user import _login
from huggingface_hub import hf_api
_login(hf_api, token=HF_Token)

print("Lucid Dreams has connected to Huggingface")
time.sleep(5)
output.clear()

from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler

if Schedule == "PNDMS":
  scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)
if Schedule == "DDIM":
  scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
if Schedule == "K-LMS":
  scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")


print(scheduler)
time.sleep(2)
output.clear()

#Import Torch and Stable Diffusion Pipeline
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

##from diffusers import StableDiffusionPipeline #Old Code
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline

#Precision Choice
if Precision == "Low Precision [Low Ram]":
  pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  
  pipeImg = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  
  pipePaint = StableDiffusionInpaintPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  

if Precision == "Higher Precision [High Ram]":
  pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, use_auth_token=True)
  pipeImg = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, use_auth_token=True)  
  pipePaint = StableDiffusionInpaintPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler, use_auth_token=True)  


#Disables NSFW Filter
def dummy(images, **kwargs): return images, False 
pipe.safety_checker = dummy
pipeImg.safety_checker = dummy

#Importing Init Image and Correcting Image
import requests
from io import BytesIO
from PIL import Image

pipe = pipe.to("cuda")
pipeImg = pipe.to("cuda")
pipePaint = pipe.to("cuda")

print("Lucid Dreams has connected to the Stable Diffusion model")
time.sleep(2)
output.clear()
print("Lucid Dreams is running on a " + str(nvidiasmi_output))

# Generate Image

In [ ]:
#@title Check Previous Batch_Name's
#@markdown This will print off your previous `Batch_Name's`

pastExports = os.listdir("/content/drive/MyDrive/003 - Creative/002 - Personal/AI Generated/Stable Diffusion/Exports/")
print("You have made the following creation batches: " + str(pastExports))

In [ ]:


#@title #Generate Image
#@markdown ### Setting `Seed` to `0` will generate a random seed. The seed will be printed and saved in your settings file.
#@markdown ---
Batch_Name = "" #@param {type:"string"}
Prompt = "" #@param {type:"string"}
intImage = "" #@param {type:"string"}
intMask = "" #@param {type:"string"}
Strength = 0.85 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
Steps = 55 #@param {type:"slider", min:10, max:150, step:5}
Guidance_Scale = 7 #@param {type:"slider", min:1, max:20, step:0.5}
Seed = 0 #@param {type:"number"}

generator = generator = torch.Generator("cuda").manual_seed(Seed)

if Prompt != "" and intImage != "":
  outputType = "prompt and image with a Strength of " + str(Strength)
if Prompt != "" and intImage == "":
  outputType = "prompt"
if Prompt == "" and intImage != "":
  outputType = "image with a Strength of " + str(Strength)

import PIL.Image

if intImage == "":
  print("Stable Diffusion: Using prompt with a Random Seed")
  if Seed == 0 or Seed == None or Seed == "":
    with autocast("cuda"):
      import random
      Seed = random.randint(1, 256872468)
      generator = generator = torch.Generator("cuda").manual_seed(Seed)
      image = pipe(Prompt, num_inference_steps=Steps, generator=generator, guidance_scale=Guidance_Scale)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
  else:
    print("Stable Diffusion: Using prompt with a specific Seed")
    with autocast("cuda"):
      image = pipe(Prompt, num_inference_steps=Steps, generator=generator)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

if intImage != "":
  init_img = PIL.Image.open(intImage).convert("RGB")
  init_img = init_img.resize((512, 512))
  init_img
  if Seed == 0 or Seed == None or Seed == "":
    with autocast("cuda"):
      import random
      Seed = random.randint(1, 256872468)
      generator = generator = torch.Generator("cuda").manual_seed(Seed)
      image = pipeImg(Prompt, init_image=init_img, strength=Strength, num_inference_steps=Steps, generator=generator, guidance_scale=Guidance_Scale)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
  else:
    with autocast("cuda"):
      image = pipeImg(Prompt, init_image=init_img, strength=Strength, num_inference_steps=Steps, generator=generator)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

if intImage != "" and intMask != "":
  init_img = PIL.Image.open(intImage).convert("RGB")
  init_img = init_img.resize((512, 512))
  init_img
  mask_image = PIL.Image.open(intMask).convert("RGB")
  mask_image = init_img.resize((512, 512))
  mask_image
  if Seed == 0 or Seed == None or Seed == "":
    print("Stable Diffusion: Using Int Image and Mask with a Random Seed")
    with autocast("cuda"):
      import random
      Seed = random.randint(1, 256872468)
      generator = generator = torch.Generator("cuda").manual_seed(Seed)
      image = pipePaint(Prompt, init_image=init_img, mask_image=mask_image, strength=Strength, num_inference_steps=Steps, generator=generator, guidance_scale=Guidance_Scale)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
  else:
    print("Stable Diffusion: Using Int Image and Mask with a specific Seed")
    with autocast("cuda"):
      image = pipePaint(Prompt, init_image=init_img, mask_image=mask_image, strength=Strength, num_inference_steps=Steps, generator=generator)["sample"][0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)


#Create temp file as backup, and quick way to use as init
image.save(f"/content/Stable-Diffusion-Temp.png")

#Create or open folder in Google Drive path based on export name.
save_image_path = "/content/drive/MyDrive/" + Save_Path  + "/Exports/" + Batch_Name
if not os.path.exists(save_image_path):
  os.mkdir(save_image_path)

#Save image based on export name, and increment the numbers from 0000 to 9999
i = 0
while os.path.exists("/content/drive/MyDrive/" + Save_Path  + "/Exports/" + Batch_Name + "/" + Batch_Name + "-%04d.png" % i):
  i += 1
incre = "-%04d" % i

#image.save(f"/content/drive/MyDrive/" + save_path + "/Exports/" + export_name + "/" + export_name + "-%04d.png" % i)
image.save(f"/content/drive/MyDrive/" + Save_Path + "/Exports/" + Batch_Name + "/" + Batch_Name + incre + ".png")
file = open(f"/content/drive/MyDrive/" + Save_Path + "/Exports/" + Batch_Name + "/" + Batch_Name + incre + ".txt", "w")
file.write("Stable Diffusion Pipeline Settings\nExport Name: " + Batch_Name + incre + "\nInit Image: " + intImage + "\nSteps: " + str(Steps) + "\nSeed: " + str(Seed) + "\nStrength: " + str(Strength) + "\nGuidance Scale: " + str(Guidance_Scale) + "\nPrecision: " + Precision + "\nSchedule: " + Schedule + "\nPrompt: " + str(Prompt))
file.close

#Display generated image in Google Colab
from IPython.display import display
display(image)
print("We generated a " + Batch_Name + " image based on a " + outputType + ". The seed used " + str(Seed) + " using " + str(Steps) + " steps with a guidance scale of " + str(Guidance_Scale) + ". The image and settings have been saved to your drive.")

# Upscaler


In [ ]:
#@title Upscale Current Export at 3X
#@markdown Inserting a path into `image_bypass` will allow you to use the upscaler without rendering an image.
image_bypass = "" #@param {type:"string"}

if image_bypass == "":
  exportToUpscale = "/content/Stable-Diffusion-Temp.png"
else:
  exportToUpscale = image_bypass
  
torch.cuda.empty_cache() 

import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

original = r'' + exportToUpscale
target = r'/content/Real-ESRGAN/upload/Stable-Diffusion-Temp.png'

shutil.copyfile(original, target)

!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --suffix "Upscaled" --outscale 3 --face_enhance
print("Finished")
time.sleep(2)
#output.clear()

from IPython.core.display import Image, display
tempUpscale = r"/content/Real-ESRGAN/results/Stable-Diffusion-Temp_Upscaled.png"
permUpscale = r"/content/drive/MyDrive/" + Save_Path + "/Exports/" + Batch_Name + "/" + Batch_Name + incre + "-Upscaled.png"

shutil.copyfile(tempUpscale, permUpscale)
print("Saving upscaled version of " + Batch_Name + incre + " to your Google Drive.")
time.sleep(2)
output.clear()

display(Image(tempUpscale))